In [2]:
import json
import re   # для парсинга ответа модели
import numpy as np
import random
import pickle

# Открываем файл и загружаем данные
with open('Kirill_dumps.json', 'r', encoding='utf-8') as file:
    data = json.load(file)  # преобразует JSON в Python-объект (dict/list)

contexts = [d['context'] for d in data]

print('Contexts:')
for c in contexts:
    print(len(c), sum(len(s) for s in c))

# какой случай к какому проекту относится
petclinic_idx = [0, 5, 10, 11, 12, 13]
keyist_eccomerce_idx = [2, 3, 4, 6, 14, 15, 16]
quarkus_idx = [1, 7, 8, 9, ]

idx2project = np.array(['']*17)
idx2project[petclinic_idx] = 'p'
idx2project[keyist_eccomerce_idx] = 'c'
idx2project[quarkus_idx] = 'q'
# idx2project

random.seed(42)

def get_context(i):
    context = contexts[i]   # контекст для текущего сценария
    N = len(context)    # число кусков

    other_project_contexts_idx = np.arange(len(contexts))[idx2project != idx2project[i]]
    other_project_contexts = [contexts[idx] for idx in other_project_contexts_idx]

    this_project_contexts_idx = np.arange(len(contexts))[idx2project == idx2project[i]]
    this_project_contexts = [contexts[idx] for idx in this_project_contexts_idx if idx != i]

    assert len(other_project_contexts) + len(this_project_contexts) + 1 == len(contexts)

    other_project_parts = []
    for c in other_project_contexts:
        other_project_parts.extend(c)

    this_project_parts = []
    for c in this_project_contexts:
        this_project_parts.extend(c)

    # res = [(p,'.') for p in context] + [(random.choice(other_project_parts), 'other') for _ in range(N)] + [(random.choice(this_project_parts), 'this') for _ in range(N)] 
    res = [(p,'.') for p in context] + [(random.choice(other_project_parts), 'other') for _ in range(N)]
    random.shuffle(res)

    return res

new_data = [{'code': d['code_under_test'], 'scenarios': d['scenarios'], 'context': get_context(i)} for i, d in enumerate(data) if i != 8]

with open('data.pkl', 'ab') as f:
    pickle.dump(new_data, f)

Contexts:
10 4029
4 1526
54 18551
32 6797
8 2816
7 2060
32 15018
4 42608
0 0
50 32733
22 7798
5 1873
15 5875
5 1570
20 5343
33 11246
30 7629
